In [2]:
!pip install selenium

In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.common.exceptions import NoSuchElementException

from difflib import SequenceMatcher
from bs4 import BeautifulSoup
############################################################
import os, csv
import time


class search_save_CSV:
    def __init__(self, i=0):
        
        header = [\
            "user-agent=Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3", \
            "user-agent=Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36", \
            "user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit 537.36 (KHTML, like Gecko) Chrome", \
        ]
        
        options = webdriver.ChromeOptions()
        # options.add_argument('headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument(header[i])
        options.add_argument('Accept-Charset=utf-8')
        
        self.driver = webdriver.Chrome("./chromedriver.exe", options=options)
        self.driver.implicitly_wait(5)
        
    def getKeyword_andOpen(self, search_text):
        self.driver.get("https://m.map.naver.com/")
        
        self.search_ = search_text
        self.driver.find_element_by_class_name("Nsearch._searchKeywordView._searchGuide").click()
        
        self.driver.find_element_by_class_name("Nbox_input_text._search_input").send_keys(self.search_)
        self.driver.find_element_by_class_name("_formSearch").submit()
        
    def errorCheck(self):
        # print(self.driver.find_element_by_class_name('interstitial-wrapper'))
        try:
            self.driver.find_element_by_class_name('interstitial-wrapper')
            print("error-reload")
            return True
        except NoSuchElementException:
            return False
        
    def search_and_save(self):
        searchList = self.driver.find_elements_by_class_name("_item._lazyImgContainer")
        choicePer, choiceIndex = 0, -1
        for i, element in enumerate(searchList):
            # print(i, ":", element.get_attribute("data-title"))    
            ratio = self.getMatchPercent(element.get_attribute("data-title"), self.search_)
            print("{0:.1f}%".format(ratio))
            
            if ratio < 50:
                pass
            else:
                if choicePer < ratio:
                    choicePer = ratio
                    choiceIndex = i
        if choiceIndex == -1:
            print("일치결과 없음")
            return False
        else:
            print("선택 index {0} : {1} 과의 일치율 {2:.1f}".format(choiceIndex, self.search_, choicePer))
            self.index = choiceIndex
            return True
        
    def get_menu_list(self): 
        elements = self.driver.find_elements_by_class_name("sp_map.btn_price._linkMenu")
        try:
            elements[self.index].click()
        except Exception:
            "가격정보 없음"
            return False
        time.sleep(5)
        
        temp = str(self.driver.current_url)
        temp = temp[:-4] + 'menu/list'
        print(temp)
        self.driver.get(temp)
        
        soup = BeautifulSoup(self.driver.page_source, 'html.parser')
        elements = soup.find_all("li", class_="_1jXsR")
        menuName = soup.find_all("span", class_="_1zr5s")
        menuPrice = soup.find_all("div", class_="_1cQ4a")

        for i, menu in enumerate(menuName):
            print(i, menu.text, ":", menuPrice[i].text)
        self.name = menuName
        self.price = menuPrice
        return True
        
    def save_to_csv(self, pk):
        saveFileName = os.path.abspath("../data/price.csv")
        if not os.path.exists(saveFileName):
            file = open(saveFileName, "w", newline="", encoding='utf-8')
            newWriter = csv.writer(file)
            newWriter.writerow(["관리번호", "메뉴이름", "가격"])
            file.close()
        file = open(saveFileName, "a", newline="", encoding='utf-8')
        writer = csv.writer(file)
        for i, menu in enumerate(self.name):
            row = [str(pk), menu.text, self.price[i].text]
            writer.writerow(row)
        file.close()      
        
        
    def getMatchPercent(self, one, two):
        s = SequenceMatcher(None, one, two)
        return s.ratio() * 100
    
    def close_driver(self):
        self.driver.quit()

In [2]:
from pyspark import SparkContext, SparkConf

conf = SparkConf().setAppName("coffee-price").setMaster("local[*]")
sc = SparkContext(conf=conf).getOrCreate()

In [3]:
import os

data_path = os.path.join("..", "data", "data.csv")
raw_data = sc.textFile(data_path)
raw_data_header = raw_data.take(1)[0]

In [4]:
headerList = str(raw_data_header).split(",") # 관리번호 = PK
need_column = ['관리번호', '영업상태구분코드', '영업상태명', '소재지전화',\
        '소재지면적','소재지전체주소', '도로명전체주소', '사업장명', '최종수정시점',\
        '업태구분명', '좌표정보(X)', '좌표정보(Y)', '시설총규모', '홈페이지']
need_column_index = []
column_list = []
for i, value in enumerate(headerList):
    if str(value) in need_column:
        need_column_index.append(i)
        column_list.append(str(value))

In [5]:
data = raw_data.filter(lambda line: line!=raw_data_header) \
    .map(lambda line: line.split(",")) \
    .filter(lambda tokens: tokens[24] == '커피숍' and tokens[7] =='영업/정상') \
    .map(lambda tokens: [tokens[i] for i in need_column_index])

In [6]:
elements = data.take(100000)
print(len(elements))

62640


In [7]:
elements = data.take(100000)
dongList = list()
num = 0
search = search_save_CSV(num)

for i, element in enumerate(elements):
    if i < 32637:
        continue
    print(i, "번째.")
#     if i%10 == 9:
#         search.close_driver()
#         time.sleep(1)
#         num = num + 1
#         search = search_save_CSV(num % 3)
    for i in range(len(column_list)):
        print("[%s]" % i, column_list[i], ":", element[i])
    continue_list = \
    ["스타벅스", "이디야", "투썸", "투썸플레이스", "엔젤리너스", "커피빈", "구스토", "달콤커피", "메가커피", \
    "바나프레소", "빽다방", "셀렉토", "요거프레소", "주커피", "드롭탑", "띠아모", "카페베네", "봄봄", \
    "커핀그루나루", "커피마마", "커피에 반하다", "탐앤탐스", "폴 바셋", "할리스", "파스쿠찌", "쥬씨"]
    
    name_list = str(element[7]).split()
    flag = False
    for comp in name_list:
        for li in continue_list:
            if li in comp:
                flag = True
                break
        if flag:
            break
    if flag: # 프랜차이즈의 경우는 넘어간다.
        continue
    
    address = str(element[5]).split()
    try:
        dongList.append(address[2])
        print("************************************************")
        search_keyword = address[2] + " " + element[7]
        
        search.getKeyword_andOpen(search_keyword)
    
        
        # check error page
        while search.errorCheck():
            time.sleep(10)
            search.close_driver()
            num = num + 1
            search = search_save_CSV(num % 3)
            search.getKeyword_andOpen(search_keyword)
            
            
        flag_ = search.search_and_save()
        if flag_:
            if search.get_menu_list():
                time.sleep(1)
                search.save_to_csv(element[0])
    except Exception:
        continue
search.close_driver()

32637 번째.
[0] 관리번호 : 3850000-104-2014-00049
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : 031 422 1880
[4] 소재지면적 : 145.20
[5] 소재지전체주소 : 경기도 안양시 동안구 관양동 1725-1번지 외1필지 스마트프라자 105  106  107호
[6] 도로명전체주소 : 경기도 안양시 동안구 동편로 6 (관양동  스마트프라자 105  106  107호 )
[7] 사업장명 : 할리스커피
[8] 최종수정시점 : 20190715101104
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 37.40568687947214
[11] 좌표정보(Y) : 126.97145223147422
[12] 시설총규모 : 145.2
[13] 홈페이지 : None
32638 번째.
[0] 관리번호 : 3850000-104-2016-00096
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : 40.28
[5] 소재지전체주소 : 경기도 안양시 동안구 관양동 1448-20번지 외2필지 1층
[6] 도로명전체주소 : 경기도 안양시 동안구 관악대로342번길 28  1층 (관양동)
[7] 사업장명 : 카페모먼트
[8] 최종수정시점 : 20200302173630
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 37.40223012986919
[11] 좌표정보(Y) : 126.960900347989
[12] 시설총규모 : 40.28
[13] 홈페이지 : None
************************************************
71.4%
71.4%
선택 index 0 : 동안구 카페모먼트 과의 일치율 71.4
https://m.place.naver.com/restaurant/1335788114/menu/list
0 아인슈페너 : 4,500원
1 아메리카노 : 3,000원
2 카페라떼 : 3,500원
3 바닐라라

32651 번째.
[0] 관리번호 : 3840000-104-2015-00025
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : 031 471 0190
[4] 소재지면적 : None
[5] 소재지전체주소 : 경기도 안양시 만안구 석수동 157-1번지
[6] 도로명전체주소 : 경기도 안양시 만안구 삼막로56번길 30  1층 (석수동)
[7] 사업장명 : 아미가
[8] 최종수정시점 : 20150421181407
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 37.42385485192807
[11] 좌표정보(Y) : 126.9140655353426
[12] 시설총규모 : 69.6
[13] 홈페이지 : None
************************************************
60.0%
50.0%
선택 index 0 : 만안구 아미가 과의 일치율 60.0
https://m.place.naver.com/restaurant/37186926/menu/list
0 더치 아메리카노 : 4,500원
1 핸드드립 커피 : 6,000원
2 생과일 쥬스 : 6,500원
3 건강음료 : 7,000원
4 차 : 5,000원
32652 번째.
[0] 관리번호 : 3850000-104-2015-00026
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : 031 386 3329
[4] 소재지면적 : None
[5] 소재지전체주소 : 경기도 안양시 동안구 관양동 1385-3번지
[6] 도로명전체주소 : 경기도 안양시 동안구 관악대로 305  지상1층 (관양동)
[7] 사업장명 : 효재커피(HYOJAE coffee)
[8] 최종수정시점 : 20150429112914
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 37.40332398720498
[11] 좌표정보(Y) : 126.95675163509188
[12] 시설총규모 : 15.89
[13] 홈페이지 : None
***********

일치결과 없음
32667 번째.
[0] 관리번호 : 3840000-104-2014-00020
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : 031 469 0591
[4] 소재지면적 : None
[5] 소재지전체주소 : 경기도 안양시 만안구 안양동 601-2번지
[6] 도로명전체주소 : 경기도 안양시 만안구 성결대학로 35-3  지상1층 (안양동)
[7] 사업장명 : 발칸카페
[8] 최종수정시점 : 20180323131941
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 37.38200428958365
[11] 좌표정보(Y) : 126.93027661893336
[12] 시설총규모 : 20.78
[13] 홈페이지 : None
************************************************
66.7%
선택 index 0 : 만안구 발칸카페 과의 일치율 66.7
https://m.place.naver.com/restaurant/904360017/menu/list
0 아메리카노 : 1,500원
1 라떼 : 2,000원
2 스무디 : 3,000원
32668 번째.
[0] 관리번호 : 3850000-104-2015-00067
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : None
[5] 소재지전체주소 : 경기도 안양시 동안구 호계동 1049번지
[6] 도로명전체주소 : 경기도 안양시 동안구 평촌대로217번길 27  107호 (호계동  백두프라자)
[7] 사업장명 : 커피에반하다 범계역2호점
[8] 최종수정시점 : 20180424100715
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 37.39022889076012
[11] 좌표정보(Y) : 126.953991862433
[12] 시설총규모 : 42
[13] 홈페이지 : None
************************************************
86.

일치결과 없음
32685 번째.
[0] 관리번호 : 3850000-104-2014-00082
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : None
[5] 소재지전체주소 : 경기도 안양시 동안구 호계동 1050번지
[6] 도로명전체주소 : 경기도 안양시 동안구 평촌대로217번길 45 (호계동  평촌코아주차빌딩 119호)
[7] 사업장명 : 키스커피 범계점
[8] 최종수정시점 : 20140821133956
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 37.38990844362843
[11] 좌표정보(Y) : 126.95348377504574
[12] 시설총규모 : 34.32
[13] 홈페이지 : None
************************************************
73.7%
선택 index 0 : 동안구 키스커피 범계점 과의 일치율 73.7
https://m.place.naver.com/restaurant/35446889/menu/list
0 아메리카노 : 3,400원
1 카페라뗴 : 4,000원
2 카푸치노 : 4,000원
3 바닐라 라떼 : 4,300원
4 헤이즐넛 라떼 : 4,300원
5 카라멜 마끼아또 : 4,500원
6 카페 모카 : 4,500원
7 민트 카페 모카 : 4,500원
8 그린티 라떼 : 4,000원
9 고구마 라떼 : 4,000원
10 블루베리 라떼 : 4,000원
11 다크 쇼콜라 : 4,300원
12 리얼 밀크티 : 4,500원
13 레몬 티 : 5,000원
14 자몽 티 : 5,000원
32686 번째.
[0] 관리번호 : 3850000-104-2014-00017
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : 0707556436
[4] 소재지면적 : None
[5] 소재지전체주소 : 경기도 안양시 동안구 호계동 920-6번지 1층
[6] 도로명전체주소 : 경기도 안양시 동안구 귀인로79번길

일치결과 없음
32701 번째.
[0] 관리번호 : 3850000-104-2020-00025
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : 43.32
[5] 소재지전체주소 : 경기도 안양시 동안구 관양동 1497번지 지상1층
[6] 도로명전체주소 : 경기도 안양시 동안구 흥안대로507번길 23  지상1층 (관양동)
[7] 사업장명 : 올래협동조합 북까페
[8] 최종수정시점 : 20200319135910
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 37.398841604854226
[11] 좌표정보(Y) : 126.97506487238189
[12] 시설총규모 : 43.32
[13] 홈페이지 : None
************************************************
일치결과 없음
32702 번째.
[0] 관리번호 : 3850000-104-2020-00035
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : None
[5] 소재지전체주소 : 경기도 안양시 동안구 호계동 985번지
[6] 도로명전체주소 : 경기도 안양시 동안구 경수대로 596  1층 (호계동)
[7] 사업장명 : 진커피하우스(Jin coffee house)
[8] 최종수정시점 : 20200323145105
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 37.37572433732376
[11] 좌표정보(Y) : 126.9568175549385
[12] 시설총규모 : 0
[13] 홈페이지 : None
************************************************
일치결과 없음
32703 번째.
[0] 관리번호 : 3840000-104-2020-00021
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : .00
[5] 소재지전체주소 : 

...

일치결과 없음
33330 번째.
[0] 관리번호 : 3860000-104-2018-00020
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : 42.51
[5] 소재지전체주소 : 경기도 부천시 소사본동 198-15번지 1층일부
[6] 도로명전체주소 : 경기도 부천시 은성로68번길 21-10  1층일부층 (소사본동)
[7] 사업장명 : 라온트리(RAON TREE)
[8] 최종수정시점 : 20180312144415
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 37.474123954218214
[11] 좌표정보(Y) : 126.79878404003978
[12] 시설총규모 : 42.51
[13] 홈페이지 : None
************************************************
33.3%
일치결과 없음
33331 번째.
[0] 관리번호 : 3860000-104-2018-00025
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : 032 344 5326
[4] 소재지면적 : 48.70
[5] 소재지전체주소 : 경기도 부천시 옥길동 744-1번지 부천옥길대우센트럴프라자 104호
[6] 도로명전체주소 : 경기도 부천시 옥길로 110-25  부천옥길대우센트럴프라자 1층 104호 (옥길동)
[7] 사업장명 : 샌드위치가맛있는옥길동커피
[8] 최종수정시점 : 20180308150547
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 37.46686340478283
[11] 좌표정보(Y) : 126.82349941896963
[12] 시설총규모 : 48.7
[13] 홈페이지 : None
************************************************
일치결과 없음
33332 번째.
[0] 관리번호 : 3860000-104-2018-00056
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 

50.0%
37.5%
60.0%
24.0%
46.2%
선택 index 2 : 괴안동 어울림 과의 일치율 60.0
33347 번째.
[0] 관리번호 : 3860000-104-2016-38701
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : 032 677 2673
[4] 소재지면적 : 7.28
[5] 소재지전체주소 : 경기도 부천시 원종동 232번지  1층일부
[6] 도로명전체주소 : 경기도 부천시 소사로794번길 6  1층 일부호 (원종동)
[7] 사업장명 : 프롬비(from B)
[8] 최종수정시점 : 20191119105939
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 37.52684051059431
[11] 좌표정보(Y) : 126.80512265437035
[12] 시설총규모 : 7.28
[13] 홈페이지 : None
************************************************
33.3%
일치결과 없음
33348 번째.
[0] 관리번호 : 3860000-104-2016-38702
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : 16.05
[5] 소재지전체주소 : 경기도 부천시 삼정동 19-11번지 외1필지
[6] 도로명전체주소 : 경기도 부천시 오정로 45 (삼정동)
[7] 사업장명 : 싸다커피(SSADA COFFEE)
[8] 최종수정시점 : 20160425163752
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 37.52607902948207
[11] 좌표정보(Y) : 126.76515877083742
[12] 시설총규모 : 16.05
[13] 홈페이지 : None
************************************************
일치결과 없음
33349 번째.
[0] 관리번호 : 3860000-104-2016-38718
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영

75.9%
선택 index 0 : 원종동 커피에반하다(부천원종점) 과의 일치율 75.9
https://m.place.naver.com/restaurant/36435869/menu/list
0 아메리카노 : 변동
33365 번째.
[0] 관리번호 : 3860000-104-2015-37217
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : 032 259 9500
[4] 소재지면적 : 41.75
[5] 소재지전체주소 : 경기도 부천시 상동 534-5번지 현해프라자 105호
[6] 도로명전체주소 : 경기도 부천시 상동로 105 (상동  현해프라자 105호)
[7] 사업장명 : 아마스빈 상동점
[8] 최종수정시점 : 20180724144240
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 37.50662371876778
[11] 좌표정보(Y) : 126.75277505784587
[12] 시설총규모 : 41.75
[13] 홈페이지 : None
************************************************
84.2%
선택 index 0 : 상동 아마스빈 상동점 과의 일치율 84.2
https://m.place.naver.com/restaurant/36646025/menu/list
0 아메리카노 : 1,800원
1 얼그레이밀크티 : 3,200원
2 오레오쉐이크 : 3,900원
3 초코버블티 : 3,500원
4 사하라 : 3,800원
5 딸기요거트 : 3,800원
6 하동녹차오레오 : 4,200원
33366 번째.
[0] 관리번호 : 3860000-104-2014-36468
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : 032 341 7710
[4] 소재지면적 : 30.10
[5] 소재지전체주소 : 경기도 부천시 역곡동 112번지 1층1호
[6] 도로명전체주소 : 경기도 부천시 부일로 752 (역곡동  1층1호)
[7] 사업장명 : 커피에반하다
[8] 최종수정시점 

81.8%
선택 index 0 : 중동 커피베이부천부흥점 과의 일치율 81.8
https://m.map.naver.com/search2/site.nhn?query=%EC%A4%91%EB%8F%99%20%EC%BB%A4%ED%94%BC%EB%B2%A0%EC%9D%B4%EB%B6%80%EC%B2%9C%EB%B6%80%ED%9D%A5%EC%A0%90&sm=hty&style=v5&code=36992825#/anchor/demenu/list
33382 번째.
[0] 관리번호 : 3860000-104-2016-38795
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : 21.39
[5] 소재지전체주소 : 경기도 부천시 고강동 284-24번지 신원프린스C동 1층 101호
[6] 도로명전체주소 : 경기도 부천시 역곡로 499 (고강동  신원프린스C동 1층 101호)
[7] 사업장명 : 플레이스38
[8] 최종수정시점 : 20190710103753
[9] 업태구분명 : 커피숍
[10] 좌표정보(X) : 37.52911746023566
[11] 좌표정보(Y) : 126.81582147960646
[12] 시설총규모 : 21.39
[13] 홈페이지 : None
************************************************
error-reload
error-reload
error-reload
75.0%
선택 index 0 : 고강동 플레이스38 과의 일치율 75.0
https://m.place.naver.com/restaurant/38264617/menu/list
0 마카롱  : 1,500원
1 아메리카노 : 2,000원
2 케이크 : 10,000원
33383 번째.
[0] 관리번호 : 3860000-104-2015-37505
[1] 영업상태구분코드 : 01
[2] 영업상태명 : 영업/정상
[3] 소재지전화 : None
[4] 소재지면적 : 25.50
[5] 소재지전체주소 : 경기도 부천시 

KeyboardInterrupt: 